# Plot Conditional Adjusted Predictions

This notebook shows how to use, and the capabilities, of the `plot_cap` function. The `plot_cap` function is a part of Bambi's sub-package `plots` that features a set of tools used to interpret complex regression models that is inspired by the R package [marginaleffects](https://vincentarelbundock.github.io/marginaleffects/articles/predictions.html#conditional-adjusted-predictions-plot). 

## Interpreting Generalized Linear Models

Generalized linear models (GLMs) are a broad family of models where the response variable $y_i$ is assumed to follow an exponential family distribution with mean $\mu_i$, which itself is assumed to be some linear function of $x_{i}^T\beta$. The mean output $\mu_i$, depending on the response distribution, is then "pushed" through a link function, which is then used as the parameter for the output distribution (likelihood). The job of the link function is often to _bound_ the parameter between some range or to make it more _plausible_.


**To do**
- generally speaking what is a GLM and GLMM
- why are GLMs and GLMMs difficult to interpet
- state the link functions
- state why link functions make the interpretation of these models more difficult
- give a short example on difficuly of parameter interpretation?

## Using `plot_cap`

**To Do**
- `plot_cap` objective
- motivation from marginaleffects

Demos:
- use demos from Tomas or Gabriel's Gist
- `pps=True` functionality
- subplots
- different groups, etc.

## Under the Hood

**To Do**
- Add explanation on what is going on in the source code
- What is `cap_data` and how is it used?
- What are the default values for computing adjusted predictions
- State what the conditonal adjusted means in _conditional adjusted predictions_